In [39]:
class Persona:
    lista = []

    def __init__(self, nombre, correo):
        self.nombre = nombre
        self.correo = correo

    def registrar(self):
        Persona.lista.append(self)
        print(f"La persona {self.nombre} ha sido registrada con el correo {self.correo}")

    def actualizar_datos(self, nombre, correo):
        self.nombre = nombre
        self.correo = correo
        print("Los datos han sido actualizados")

    @classmethod
    def personas_registradas(cls):
        print("Personas registradas")
        for persona in cls.lista:
            print(f"-{persona.nombre} - {persona.correo}")

class Usuario(Persona):
    def __init__(self, nombre, correo):
        super().__init__(nombre, correo)
        self.historial_reservas = []
        self.historial_compras = []

    def reservar(self, funcion, asientos):
        if asientos <= 0:
            print("El número de asientos debe ser mayor que cero.")
            return
        if asientos <= funcion.asientos_disponibles:
            funcion.asientos_disponibles -= asientos
            reserva = Reserva(self, funcion, asientos)
            self.historial_reservas.append(reserva)
            print(f"Reserva realizada para '{funcion.pelicula.titulo}' en la sala {funcion.sala.identificador}.")
        else:
            print("No hay suficientes asientos disponibles.")

    def cancelar_reserva(self, funcion):
        reserva = next((r for r in self.historial_reservas if r.funcion == funcion), None)
        if reserva:
            funcion.asientos_disponibles += reserva.asientos
            self.historial_reservas.remove(reserva)
            print(f"Reserva cancelada para '{funcion.pelicula.titulo}'.")
        else:
            print("No tienes una reserva para esta función.")

    def comprar_producto(self, zona_comida, producto, cantidad):
        if producto in zona_comida.productos_disponibles:
            stock_disponible = zona_comida.productos_disponibles[producto]["stock"]
            precio_base = zona_comida.productos_disponibles[producto]["precio"]

            if stock_disponible >= cantidad:
                total = precio_base * cantidad
                if zona_comida.promocion and zona_comida.promocion.tipo == "comida":
                    total = zona_comida.promocion.aplicar_descuento(total)

                zona_comida.productos_disponibles[producto]["stock"] -= cantidad  # 🔹 Reducir stock correctamente
                self.historial_compras.append((producto, cantidad, total))
                print(f"{self.nombre} compró {cantidad} {producto}(s) por un total de ${total:.2f}")
            else:
                print(f"No hay suficiente stock de {producto}. Disponibles: {stock_disponible}.")
        else:
            print(f"El producto {producto} no está disponible en esta zona.")



roles_validos = ["Taquillero", "Administrador"]

class Empleado(Persona):
    def __init__(self, nombre, correo, rol):
        super().__init__(nombre, correo)
        self.rol = rol

    def agregar_funcion(self, funcion):
        if self.rol not in roles_validos:
            print(f"El rol ingresado no tiene acceso a este movimiento")
        else:
            if funcion in Funcion.lista_funciones:
                print(f"La función de '{funcion.pelicula.titulo}' ya está en la cartelera y no se agregará nuevamente.")
            else:
                print(f"Función agregada correctamente: {funcion.pelicula.titulo} a las {funcion.hora} en la sala {funcion.sala.identificador}.")

    def modificar_promocion(self, promocion, nuevo_descuento, nuevas_condiciones):
        if self.rol not in roles_validos:
            print(f"El rol ingresado no tiene acceso a este movimiento")
        else:
            promocion.descuento = nuevo_descuento
            promocion.condiciones = nuevas_condiciones

    def reabastecer_producto(self, zona_comida, producto, cantidad):
        if self.rol not in roles_validos:
            print(f"El rol {self.rol} no tiene permisos para reabastecer productos.")
        else:
            if producto in zona_comida.productos_disponibles:
                zona_comida.productos_disponibles[producto]["stock"] += cantidad
                print(f"Se han añadido {cantidad} unidades de {producto}. Nuevo stock: {zona_comida.productos_disponibles[producto]["stock"]}")
            else:
                print(f"El producto {producto} no pertenece a esta zona de comida.")

class Espacio:
    def __init__(self, capacidad, identificador):
        self.capacidad = capacidad
        self.identificador = identificador

    def descripcion(self):
        return f"Espacio ID: {self.identificador}, Capacidad: {self.capacidad} personas."

class Sala(Espacio):
    def __init__(self, capacidad, identificador, tipo):
        super().__init__(capacidad, identificador)
        self.tipo = tipo
        self.disponibilidad = True
    
class ZonaComida(Espacio):
    def __init__(self, capacidad, identificador, productos_disponibles):
        super().__init__(capacidad, identificador)
        self.productos_disponibles = {producto: {"stock": 10, "precio": 50} for producto in productos_disponibles}
        self.promocion = None  # Agregar atributo promoción

    def asignar_promocion(self, promocion):
        if promocion.tipo == "comida":
            self.promocion = promocion
            print(f"Se ha asignado una promoción del {promocion.descuento}% en la zona de comida.")
        super().__init__(capacidad, identificador)
        self.productos_disponibles = {producto: {"stock": 10, "precio": 50} for producto in productos_disponibles}

class Pelicula:
    def __init__(self, titulo, genero, duracion, clasificacion):
        self.titulo = titulo
        self.genero = genero
        self.duracion = duracion
        self.clasificacion = clasificacion
        
    def detalles(self):
        return f"Título: {self.titulo}, Género: {self.genero}, Duración: {self.duracion} min, Clasificación: {self.clasificacion}."

class Funcion:
    lista_funciones = []

    def __init__(self, pelicula, sala, hora):
        self.pelicula = pelicula
        self.sala = sala
        self.hora = hora
        self.asientos_disponibles = sala.capacidad
        
        if not any(f.pelicula.titulo == self.pelicula.titulo and f.sala.identificador == self.sala.identificador and f.hora == self.hora for f in Funcion.lista_funciones):
            Funcion.lista_funciones.append(self)
    
    @classmethod
    def mostrar_cartelera(cls):
        print("Cartelera del Cine:")
        for funcion in cls.lista_funciones:
            print(f"Película: {funcion.pelicula.titulo} | Sala: {funcion.sala.identificador} | Hora: {funcion.hora} | Asientos disponibles: {funcion.asientos_disponibles}")

class Reserva:
    def __init__(self, usuario, funcion, asientos):
        self.usuario = usuario
        self.funcion = funcion
        self.asientos = asientos

class Promocion:
    def __init__(self, descuento, condiciones, tipo="general"):
        self.descuento = descuento
        self.condiciones = condiciones
        self.tipo = tipo
    
    def aplicar_descuento(self, precio_original):
        return precio_original * (1 - self.descuento / 100)
    
    def mostrar(self):
        print(f"Promoción: {self.descuento}% de descuento. Condiciones: {self.condiciones}")

Probar el programa:

In [40]:
#Crear películas
pelicula1 = Pelicula("Intensamente 2", "Infantil", 169, "Todo público")
pelicula2 = Pelicula("El Conjuro", "Terror", 175, "Mayores de 18 años")

# Crear salas
sala1 = Sala(50, "Sala 1", "IMAX")
sala2 = Sala(30, "Sala 2", "3D")

# Crear funciones
funcion1 = Funcion(pelicula1, sala1, "15:00")
funcion2 = Funcion(pelicula2, sala2, "18:30")

# Mostrar cartelera
Funcion.mostrar_cartelera()

# Crear usuarios
usuario1 = Usuario("Lidia Sánchez", "lidi@gmail.com")
usuario2 = Usuario("Perla Morales", "perlaM@gmail.com")

#reserva de funciones
usuario1.reservar(funcion1, 2)
usuario2.reservar(funcion2, 5)

# Cancelar  reserva
usuario1.cancelar_reserva(funcion1)

# Crear empleado
empleado1 = Empleado("José Martínez", "josM@gmail.com", "Taquillero")

# Agregar una nueva función con el empleado
nueva_funcion = Funcion(Pelicula("iron Man", "Acción", 136, "Mayores de 12 años"), sala1, "22:00")
empleado1.agregar_funcion(nueva_funcion)

# Mostrar cartelera
Funcion.mostrar_cartelera()

# Crear promoción
promocion1 = Promocion(20, "Descuento para funciones después de las 22:00")
promocion1.mostrar()

# Modificar la promoción desde un empleado
empleado1.modificar_promocion(promocion1, 30, "Descuento para funciones IMAX")
promocion1.mostrar()

# Crear una zona de comida
zona_comida1 = ZonaComida(10, "Zona Snacks", ["Palomitas", "Refresco", "Nachos"])

usuario2.comprar_producto(zona_comida1, "Palomitas", 2)

Cartelera del Cine:
Película: Intensamente 2 | Sala: Sala 1 | Hora: 15:00 | Asientos disponibles: 50
Película: El Conjuro | Sala: Sala 2 | Hora: 18:30 | Asientos disponibles: 30
Reserva realizada para 'Intensamente 2' en la sala Sala 1.
Reserva realizada para 'El Conjuro' en la sala Sala 2.
Reserva cancelada para 'Intensamente 2'.
La función de 'iron Man' ya está en la cartelera y no se agregará nuevamente.
Cartelera del Cine:
Película: Intensamente 2 | Sala: Sala 1 | Hora: 15:00 | Asientos disponibles: 50
Película: El Conjuro | Sala: Sala 2 | Hora: 18:30 | Asientos disponibles: 25
Película: iron Man | Sala: Sala 1 | Hora: 22:00 | Asientos disponibles: 50
Promoción: 20% de descuento. Condiciones: Descuento para funciones después de las 22:00
Promoción: 30% de descuento. Condiciones: Descuento para funciones IMAX
Perla Morales compró 2 Palomitas(s) por un total de $100.00
